In [ ]:
# https://youtube-rank.com/board/bbs/board.php?bo_table=youtube
# 순위, 카테고리, 채널 , 구독자수, 조회수, 비디오수 
# 셀레늄으로 해당페이지를 열고 + BS4 를 사용해서  클롤링 10페이지

# YouTube Ranking

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)

In [3]:
# soup
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

In [4]:
tr= trs[0]

In [8]:
#순위
tr.select_one('.rank').get_text().strip()
#카테고리
tr.select_one('.category').get_text().strip()[1:-1]
#채널명
tr.select_one('.subject a').get_text().strip()
# 구독자 조회수 비디오갯수
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()

In [9]:
subscriber, view, video

('8680만', '303억4016만', '505개')

In [10]:
# 첫페이지 크롤링
lines = []
for tr in trs:
    rank = tr.select_one('.rank').get_text().strip()
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject a').get_text().strip()
    subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
    view = tr.select_one('.view_cnt').get_text().strip()
    video = tr.select_one('.video_cnt').get_text().strip()
    lines.append([rank, category, channel, subscriber, view, video])

In [13]:
df = pd.DataFrame(lines, columns=['순위', '카테고리', '채널' , '구독자수', '조회수', '비디오수'])
df.tail()

,순위,카테고리,채널,구독자수,조회수,비디오수
95,96,키즈/어린이,뽀로로(Pororo),485만,68억3934만,"4,151개"
96,97,애완/반려동물,SBS TV동물농장x애니멀봐,483만,46억2878만,"4,147개"
97,98,키즈/어린이,CoCosToy 꼬꼬스토이,477만,29억7937만,684개
98,99,음식/요리/레시피,Cooking tree 쿠킹트리,476만,4억8734만,"1,453개"
99,100,TV/방송,JTBC Drama,473만,52억2976만,"27,723개"


In [14]:
# 숫자로 바꿔주는 함수
def convert_unit(s):
    s = s.replace("억", "").replace("개", "").replace(',', '').replace('만', "0000")
    return f'{int(s):,d}'

In [15]:
convert_unit('303억4016만')

'30,340,160,000'

In [16]:
for column in df.columns[3:]:
    df[column] = df[column].apply(convert_unit)
df.head()

,순위,카테고리,채널,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"86,800,000","30,340,160,000",505
1,2,음악/댄스/가수,BANGTANTV,"74,300,000","20,078,950,000","2,174"
2,3,음악/댄스/가수,HYBE LABELS,"70,100,000","27,213,060,000","1,142"
3,4,음악/댄스/가수,SMTOWN,"31,600,000","26,746,800,000","4,108"
4,5,음악/댄스/가수,JYP Entertainment,"26,700,000","19,236,390,000","1,662"


In [17]:
from tqdm.notebook import tqdm

In [19]:
lines = []
for page in tqdm(range(1, 11)):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('tr.aos-init')
    
    for tr in trs:
        rank = tr.select_one('.rank').get_text().strip()
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject a').get_text().strip()
        subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
        view = tr.select_one('.view_cnt').get_text().strip()
        video = tr.select_one('.video_cnt').get_text().strip()
        lines.append([rank, category, channel, subscriber, view, video])

  0%|          | 0/10 [00:00<?, ?it/s]

In [20]:
df = pd.DataFrame(lines, columns=['순위', '카테고리', '채널' , '구독자수', '조회수', '비디오수'])
for column in df.columns[3:]:
    df[column] = df[column].apply(convert_unit)

In [21]:
df.head()

,순위,카테고리,채널,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"86,800,000","30,340,160,000",505
1,2,음악/댄스/가수,BANGTANTV,"74,300,000","20,078,950,000","2,174"
2,3,음악/댄스/가수,HYBE LABELS,"70,100,000","27,213,060,000","1,142"
3,4,음악/댄스/가수,SMTOWN,"31,600,000","26,746,800,000","4,108"
4,5,음악/댄스/가수,JYP Entertainment,"26,700,000","19,236,390,000","1,662"


In [22]:
df.tail()

,순위,카테고리,채널,구독자수,조회수,비디오수
995,991,BJ/인물/연예인,bobaepapa - 보배아빠,"730,000","348,400,000",393
996,996,음악/댄스/가수,Crush,"720,000","393,180,000",131
997,1000,미분류,백크 Baekk,"720,000","455,250,000","1,102"
998,999,뉴스/정치/사회,국방TV,"720,000","757,890,000","12,681"
999,998,TV/방송,KBS COMEDY: 크큭티비,"720,000","1,026,260,000","7,921"


In [23]:
df.to_csv('youtube랭킹.csv', index=False)